In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [3]:
Sample_output= pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')
print (Sample_output[:10])

  PassengerId  Transported
0     0013_01        False
1     0018_01        False
2     0019_01        False
3     0021_01        False
4     0023_01        False
5     0027_01        False
6     0029_01        False
7     0032_01        False
8     0032_02        False
9     0033_01        False


In [4]:
train_csv = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_csv = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [5]:
print(train_csv[:5])

  PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S  TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S  TRAPPIST-1e  16.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True  


In [6]:
train_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


Droping Name row as It will not have any effects on prediction

In [7]:
train_csv = train_csv.drop(['Name','Cabin','RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'],axis = 1)
print(train_csv.columns)

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP',
       'Transported'],
      dtype='object')


In [8]:
for i in train_csv.columns:
    if( train_csv.dtypes[i] == object ):
        train = train_csv[i].fillna(method= 'bfill',inplace=True,axis = 0)  


In [9]:
train_csv['Age'].describe()
train_csv['Age'] = train_csv['Age'].fillna(28.827930)

In [10]:
train_csv = pd.get_dummies(train_csv, columns = ['HomePlanet', 'Destination'])



In [11]:
train_csv.rename(columns = {'Destination_PSO J318.5-22':'PSO','Destination_TRAPPIST-1e':'TRAPPIST',
                                        'HomePlanet_Earth':'Earth','HomePlanet_Europa':'Europa',
                                        'HomePlanet_Mars':'Mars','Destination_55 Cancri e':'Cancrie'}, 
                                        inplace = True)

In [12]:
train_csv['Transported'] = train_csv['Transported'].astype(int)
train_csv['CryoSleep'] = train_csv['CryoSleep'].astype(int)
train_csv['Age'] = train_csv['Age'].astype(int)
train_csv['Earth'] = train_csv['Earth'].astype(int)
train_csv['Europa'] = train_csv['Europa'].astype(int)
train_csv['Mars'] = train_csv['Mars'].astype(int)
train_csv['PSO'] = train_csv['PSO'].astype(int)
train_csv['TRAPPIST'] = train_csv['TRAPPIST'].astype(int)
train_csv['VIP'] = train_csv['VIP'].astype(int)
train_csv['Cancrie'] = train_csv['Cancrie'].astype(int)
train_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  8693 non-null   object
 1   CryoSleep    8693 non-null   int64 
 2   Age          8693 non-null   int64 
 3   VIP          8693 non-null   int64 
 4   Transported  8693 non-null   int64 
 5   Earth        8693 non-null   int64 
 6   Europa       8693 non-null   int64 
 7   Mars         8693 non-null   int64 
 8   Cancrie      8693 non-null   int64 
 9   PSO          8693 non-null   int64 
 10  TRAPPIST     8693 non-null   int64 
dtypes: int64(10), object(1)
memory usage: 747.2+ KB


In [13]:
print(train_csv)

     PassengerId  CryoSleep  Age  VIP  Transported  Earth  Europa  Mars  \
0        0001_01          0   39    0            0      0       1     0   
1        0002_01          0   24    0            1      1       0     0   
2        0003_01          0   58    1            0      0       1     0   
3        0003_02          0   33    0            0      0       1     0   
4        0004_01          0   16    0            1      1       0     0   
...          ...        ...  ...  ...          ...    ...     ...   ...   
8688     9276_01          0   41    1            0      0       1     0   
8689     9278_01          1   18    0            0      1       0     0   
8690     9279_01          0   26    0            1      1       0     0   
8691     9280_01          0   32    0            0      0       1     0   
8692     9280_02          0   44    0            1      0       1     0   

      Cancrie  PSO  TRAPPIST  
0           0    0         1  
1           0    0         1  
2     

In [14]:
x = np.array(train_csv.drop(['PassengerId','Transported'],axis=1))
y = np.array(train_csv['Transported'])

In [15]:
x_cv, x_train, y_cv, y_train = train_test_split(x,y,test_size=0.20, random_state=1)


In [16]:
xgb_model = XGBClassifier(n_estimators = 500, learning_rate = 0.1,verbosity = 1, random_state = 0)
xgb_model.fit(x_train,y_train, eval_set = [(x_cv,y_cv)])

[0]	validation_0-logloss:0.66740
[1]	validation_0-logloss:0.64680
[2]	validation_0-logloss:0.62964
[3]	validation_0-logloss:0.61548
[4]	validation_0-logloss:0.60371
[5]	validation_0-logloss:0.59423
[6]	validation_0-logloss:0.58565
[7]	validation_0-logloss:0.57853
[8]	validation_0-logloss:0.57254
[9]	validation_0-logloss:0.56774
[10]	validation_0-logloss:0.56379
[11]	validation_0-logloss:0.56028
[12]	validation_0-logloss:0.55742
[13]	validation_0-logloss:0.55522
[14]	validation_0-logloss:0.55329
[15]	validation_0-logloss:0.55170
[16]	validation_0-logloss:0.55061
[17]	validation_0-logloss:0.54975
[18]	validation_0-logloss:0.54902
[19]	validation_0-logloss:0.54839
[20]	validation_0-logloss:0.54819
[21]	validation_0-logloss:0.54796
[22]	validation_0-logloss:0.54771
[23]	validation_0-logloss:0.54796
[24]	validation_0-logloss:0.54782
[25]	validation_0-logloss:0.54777
[26]	validation_0-logloss:0.54788
[27]	validation_0-logloss:0.54810
[28]	validation_0-logloss:0.54841
[29]	validation_0-loglos

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=500,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [17]:
xgb_model.best_iteration

499

In [18]:
accuracy_score(xgb_model.predict(x_train),y_train)

0.7941345600920069